<a href="https://colab.research.google.com/github/reynerfernaldi/LazadaScrape1/blob/main/A191690_Assignment_1_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Scraping : Lazada

### ASSIGNMENT-1-PART2 
NAME: Reyner Fernaldi<br>
MATRIC NO: A191690<br>

In [ ]:
# Install chromium, web driver and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Fetched 264 kB in 2s (168 kB/s)
Reading p

In [ ]:
import time
import re
import csv
import pandas as pd
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Open browser by webdrive
driver = webdriver.Chrome('chromedriver', options = options)

# To store product
products_link = []
products_comment=[]

urls = ["https://www.lazada.com.my/shop-smartwatches/?clickTrackInfo=58832903-7518-40c6-afa7-16bfe5a3fb57__10100415__626040162__static__0.09751687221196013__299225__7253&from=hp_categories&item_id=626040162&page=1", 
       "https://www.lazada.com.my/shop-smartwatches/?clickTrackInfo=58832903-7518-40c6-afa7-16bfe5a3fb57__10100415__626040162__static__0.09751687221196013__299225__7253&from=hp_categories&item_id=626040162&page=2", 
       "https://www.lazada.com.my/shop-smartwatches/?clickTrackInfo=58832903-7518-40c6-afa7-16bfe5a3fb57__10100415__626040162__static__0.09751687221196013__299225__7253&from=hp_categories&item_id=626040162&page=3", 
       "https://www.lazada.com.my/shop-smartwatches/?clickTrackInfo=58832903-7518-40c6-afa7-16bfe5a3fb57__10100415__626040162__static__0.09751687221196013__299225__7253&from=hp_categories&item_id=626040162&page=4"]

for url in urls:
  try:
    driver.get(url)

    RemainDue = WebDriverWait(driver, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "#root")))
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    for item in soup.findAll('div', class_='Bm3ON'):
      
      #get item links
      link = item.find('a', href=True)

      #get total of comment
      comment = item.find('span', class_='qzqFw')

      #Check if there's comment in the product
      if comment!=None :
        products_comment.append(comment.text)
      else:
        comment = '(0)'
        products_comment.append(comment)

      products_link.append(link['href'])
      
    time.sleep(2)
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".ant-pagination-item-link > button"))).click()
    time.sleep(3)
  
  except:
    RemainDue = "BLANK"

driver.close()

# Saving scraped data into dataframe

In [ ]:
#Creating a dataframe
df = pd.DataFrame({"Product Links":products_link, "Number of Comments":products_comment})

In [ ]:
#Raw dataframe
df

,Product Links,Number of Comments
0,//www.lazada.com.my/products/lemfo-lem-t-4g-28...,(1)
1,//www.lazada.com.my/products/huawei-watch-gt-2...,(297)
2,//www.lazada.com.my/products/huawei-band-6-sma...,(3836)
3,//www.lazada.com.my/products/realme-band-2-1-t...,(347)
4,//www.lazada.com.my/products/huawei-watch-3-sm...,(29)
...,...,...
116,//www.lazada.com.my/products/new-m6-jam-pintar...,(82)
117,//www.lazada.com.my/products/malaysia-set-huaw...,(98)
118,//www.lazada.com.my/products/t13-smart-watch-s...,(1)
119,//www.lazada.com.my/products/outtobe-smartwatc...,(3)


In [ ]:
# FIRST DATA PREPROCESSING
#Check if there's null values
df.isnull().sum()

Product Links         0
Number of Comments    0
dtype: int64

In [ ]:
# SECOND DATA PREPROCESSING
#Check if any missing data
df.isnull().values.any()

False

In [ ]:
# THIRD DATA PREPROCESSING
#Check if there's any duplicated data on the link
duplicate = df[df.duplicated()]
print("Duplicate Rows :")
duplicate

Duplicate Rows :


,Product Links,Number of Comments


In [ ]:
# FOURTH DATA PREPROCESSING
#Formating the data
#Remove "//"
df['Product Links'] = df['Product Links'].str.replace('//','')
#Remove "(" and ")"
df['Number of Comments'] = df['Number of Comments'].str.replace('(','').str.replace(')','')

<ipython-input-204-c2f6139a7931>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['Number of Comments'] = df['Number of Comments'].str.replace('(','').str.replace(')','')


In [ ]:
#Cleaned Data
df

,Product Links,Number of Comments
0,www.lazada.com.my/products/lemfo-lem-t-4g-286-...,1
1,www.lazada.com.my/products/huawei-watch-gt-2-p...,297
2,www.lazada.com.my/products/huawei-band-6-smart...,3836
3,www.lazada.com.my/products/realme-band-2-1-to-...,347
4,www.lazada.com.my/products/huawei-watch-3-smar...,29
...,...,...
116,www.lazada.com.my/products/new-m6-jam-pintar-s...,82
117,www.lazada.com.my/products/malaysia-set-huawei...,98
118,www.lazada.com.my/products/t13-smart-watch-sma...,1
119,www.lazada.com.my/products/outtobe-smartwatch-...,3


In [ ]:
#Check dataframe datatypes
df.dtypes

Product Links         object
Number of Comments    object
dtype: object

In [ ]:
# FIFTH DATA PREPROCESSING
#Change the 'Number of Comments' column into integer data type
df['Number of Comments']=df['Number of Comments'].astype("int")

In [ ]:
#Check the data type
df.dtypes

Product Links         object
Number of Comments     int64
dtype: object

In [ ]:
# SIXTH DATA PREPROCESSING
#Sort the data based on the number of comments
df = df.sort_values(by=['Number of Comments'], ascending=False)

In [ ]:
#Sorted Data
df

,Product Links,Number of Comments
2,www.lazada.com.my/products/huawei-band-6-smart...,3836
16,www.lazada.com.my/products/huawei-watch-gt-2-4...,1844
46,www.lazada.com.my/products/huawei-watch-fit-sm...,1178
13,www.lazada.com.my/products/realme-watch-2-1-to...,975
48,www.lazada.com.my/products/huawei-band-7-smart...,954
...,...,...
38,www.lazada.com.my/products/malaysia-set-huawei...,0
28,www.lazada.com.my/products/huawei-band-6-amber...,0
27,www.lazada.com.my/products/huawei-huawei-watch...,0
22,www.lazada.com.my/products/huawei-watch-gt-3-p...,0


In [ ]:
# SEVENTH DATA PREPROCESSING
#Reset the index
df = df.reset_index(drop=True)

In [ ]:
df

,Product Links,Number of Comments
0,www.lazada.com.my/products/huawei-band-6-smart...,3836
1,www.lazada.com.my/products/huawei-watch-gt-2-4...,1844
2,www.lazada.com.my/products/huawei-watch-fit-sm...,1178
3,www.lazada.com.my/products/realme-watch-2-1-to...,975
4,www.lazada.com.my/products/huawei-band-7-smart...,954
...,...,...
116,www.lazada.com.my/products/malaysia-set-huawei...,0
117,www.lazada.com.my/products/huawei-band-6-amber...,0
118,www.lazada.com.my/products/huawei-huawei-watch...,0
119,www.lazada.com.my/products/huawei-watch-gt-3-p...,0


In [ ]:
# EIGHTH DATA PREPROCESSING
#Adding Ranks
import numpy as np

df['Ranks'] = np.arange(1, len(df) + 1)

In [ ]:
#Final Data
df

,Product Links,Number of Comments,Ranks
0,www.lazada.com.my/products/huawei-band-6-smart...,3836,1
1,www.lazada.com.my/products/huawei-watch-gt-2-4...,1844,2
2,www.lazada.com.my/products/huawei-watch-fit-sm...,1178,3
3,www.lazada.com.my/products/realme-watch-2-1-to...,975,4
4,www.lazada.com.my/products/huawei-band-7-smart...,954,5
...,...,...,...
116,www.lazada.com.my/products/malaysia-set-huawei...,0,117
117,www.lazada.com.my/products/huawei-band-6-amber...,0,118
118,www.lazada.com.my/products/huawei-huawei-watch...,0,119
119,www.lazada.com.my/products/huawei-watch-gt-3-p...,0,120


In [ ]:
#Export to CSV
df.to_csv('Most Commented(Rated) Smartwatch Links(120 records).csv')

### ASSIGNMENT-1-PART2 
NAME: Reyner Fernaldi<br>
MATRIC NO: A191690<br>